This will show examples using the **odbc + DBI** packages and additional examples using the older **RODBC** package.  

**odbc + DBI**: How to Use the New R ODBC Package to Connect to an Oracle Database  
https://blog.redpillanalytics.com/how-to-use-the-new-r-odbc-package-to-connect-to-an-oracle-database-70ce6783e397  

**odbc**: Connect to ODBC Compatible Databases (using the DBI Interface)  
https://cran.r-project.org/web/packages/odbc/index.html  
**DBI**: R Database Interface  
https://cran.r-project.org/web/packages/DBI/index.html  

**RODBC**: ODBC Database Access  
https://cran.r-project.org/web/packages/RODBC/index.html  


In [1]:
version

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          4                           
minor          1.3                         
year           2022                        
month          03                          
day            10                          
svn rev        81868                       
language       R                           
version.string R version 4.1.3 (2022-03-10)
nickname       One Push-Up                 

In [2]:
# NOTE: the error occurs because "C:/ProgramData/" is not writable
#    The solution is to make "C:/ProgramData/Anaconda3" (and all subfolders) writable
install.packages('odbc')  

Warning message in install.packages("odbc"):
"'lib = "C:/ProgramData/Anaconda3/envs/conda_env_2023-0309/Lib/R/library"' is not writable"


ERROR: Error in install.packages("odbc"): unable to install packages


In [4]:
install.packages('odbc')

Warning message in install.packages("odbc"):
"'lib = "C:/ProgramData/Anaconda3/envs/conda_env_2023-0309/Lib/R/library"' is not writable"


ERROR: Error in install.packages("odbc"): unable to install packages


In [ ]:
library(odbc)
library(dplyr)
library(dbplyr)
# Connect using the DSN
db <- DBI::dbConnect(odbc::odbc(), "SQL")

In [ ]:
# Connect without a DSN
db <- DBI::dbConnect(odbc::odbc(),
                     Driver = 'ODBC Driver 13 for SQL Server',
                     Server = 'localhost\\SQLEXPRESS',
                     Database = "master",
                     trusted_connection = 'yes',
                     Port = 1433
                     )